In [17]:
import xmltodict
import pandas as pd

In [12]:
#read all data from current incidents.xml to get a pair of DataFrame: incdent_keys incdent_values 
def read_incident():
    with open('./Data/XMLs/incidents.xml') as fd:
        doc = xmltodict.parse(fd.read())
    keys_len1=len(list(list(doc.items())[0][1].items())[1][1])
    aa=([[list(list(list(doc.items())[0][1].items())[1][1][num_dict].items())[i][0]  
      for i in range(len(list(list(list(doc.items())[0][1].items())[1][1][num_dict].items())))] 
     for num_dict in range(keys_len1)])
    bb=([[list(list(list(doc.items())[0][1].items())[1][1][num_dict].items())[i][1]  
      for i in range(len(list(list(list(doc.items())[0][1].items())[1][1][num_dict].items())))] 
     for num_dict in range(keys_len1)])
    incdent_keys=pd.DataFrame(aa)
    incdent_values=pd.DataFrame(bb)
    return incdent_keys,incdent_values
    
incdent_keys,incdent_values=read_incident()
incdent_keys;
incdent_values;


In [13]:
#reorder the data and add to the csv file
def Reorder_incident(incdent_values,incdent_keys):
##use two steps to reorder&clean the current incidence data
#(1) import good value types into the DataFrame
        long_list=["-91","-92","-93","-94","-95","-96","-97"]
        lat_list=["41","42","43","44","45","46","47","48"]
        weekdays= ["Mon" ,"Tue", "Wed","Thur","Fri","Sat","Sun"]
        type_list=["INCIDENT"]
        direction_list=["EB","SB","WB","NB"]

        def funcLon(x):
            if isinstance(x,str):
                return x.split('.')[0] in long_list
            else:
                return False
        def funcLat(x):
            if isinstance(x,str):
                return x.split('.')[0] in lat_list
            else:
                return False
        def func_incident_type(x):
            if isinstance(x,str):
                return x.split('_')[0] in type_list
            else:
                return False
        def func_address(x):
            if isinstance(x,str):
                return x.startswith('@') 
            else:
                return False

        def func_time(x):
            if isinstance(x,str):
                return x.split()[0] in weekdays 
            else:
                return False
        def func_direction(x):
            if isinstance(x,str):
                return x in  direction_list 
            else:
                return False
            
        funcs=[funcLon,funcLat,func_time,func_incident_type,func_address,func_direction]
        columns_new=["Lon","Lat","time","INCIDENT","address","Direction"]   
        for col_name in columns_new:
            incdent_values[col_name]=None    

        func_n=0
        for col_name in columns_new:
            for i in range(len(incdent_values)):
                v=incdent_values.loc[i,:]
                try:
                    incdent_values.loc[i,col_name]=v[v.apply(funcs[func_n])].iloc[0]
                except (IndexError,ValueError) as e:
                    pass
            func_n=func_n+1
        incdent_values["Lon"]=pd.to_numeric(incdent_values["Lon"])
        incdent_values["Lat"]=pd.to_numeric(incdent_values["Lat"]) 
        
#(2)import correctly keyed values ito the DataFrame:     
        def func_Road(x):
            if isinstance(x,str):
                return x in ["@ROAD","@road","Road"]
            else:
                return False
        def func_Name(x):
            if isinstance(x,str):
                return x in ["@name"]
            else:
                return False
        funcs2=[func_Road,func_Name]
        columns_new2=["Road","Name"]
        for col_name in columns_new2:
            incdent_values[col_name]=None 
            
        func_n=0
        for col_name in columns_new2:
            for i in range(len(incdent_values)):
                v=incdent_keys.loc[i,:]
                vv=incdent_values.loc[i,:]
                try:
                    incdent_values.loc[i,col_name]=vv[v[v.apply(funcs2[func_n])].index[0]]
                except (IndexError,ValueError) as e:
                    pass
            func_n=func_n+1      
        return incdent_values[columns_new+columns_new2]
    
incdents_df=Reorder_incident(incdent_values,incdent_keys)
incdents_df.head()
# add the newly cleaned incident data to csv file
print("Incident Data Parsed and added to csv file")
with open('Data/incidence_data.csv', 'a') as f:
    incdents_df.to_csv(f, header=False)

,Lon,Lat,time,INCIDENT,address,Direction,Road,Name
0,-94.21060,45.49869,Sun Oct 07 20:03:12 CDT 2018,INCIDENT_CRASH,None,WB,I-94,2018100720031226
1,-93.33279,45.07604,Sun Oct 07 19:13:02 CDT 2018,INCIDENT_CRASH,@ Brooklyn Blvd,WB,I-94,L004_9956092
2,-93.06811,44.92538,Sun Oct 07 21:11:12 CDT 2018,INCIDENT_STALL,@ Concord St,NB,T.H.52,L004_9956228
3,-93.18790,45.01456,Sat Oct 06 10:33:29 CDT 2018,INCIDENT_ROADWORK,@ Cleveland Ave,SB,I-35W,2018100610332993
4,-93.29591,44.64518,Sun Oct 07 21:15:42 CDT 2018,INCIDENT_HAZARD,@ 210th St,NB,I-35,L004_9956232


Incident Data Parsed and added to csv file


In [48]:
#Now we can further eleminite the duplicate reports  to clean the data
#crash_df=pd.read_csv('./Data/incidence_data.csv')

In [18]:
pd.read_csv("Data/crash_data.csv")

,Unnamed: 0,Name,Date,DirectionLocation,Road,Unnamed: 5,Event
0,8,EB,I-694,@ Mississippi River,HAZARD,NaN,NaN
1,1,2018100217261325,Tue Oct 02 17:26:13 CDT 2018,SB,I-35W,NaN,ROADWORK
2,2,2018100218151677,Tue Oct 02 18:15:16 CDT 2018,SB,I-35W,@ Co Rd E,ROADWORK
3,3,2018100218161579,Tue Oct 02 18:16:15 CDT 2018,NB,I-35W,@ Co Rd J,ROADWORK
4,4,2018100219595578,Tue Oct 02 19:59:55 CDT 2018,SB,I-35W,NaN,ROADWORK
5,5,2018100219492669,Tue Oct 02 19:49:26 CDT 2018,EB,I-94,@ T.H.280,ROADWORK
6,6,2018100301051236,Wed Oct 03 01:05:12 CDT 2018,SB,I-35,NaN,CRASH
7,7,L004_9946476,Tue Oct 02 23:01:32 CDT 2018,SB,T.H.77,@ Killebrew Dr,CRASH
8,8,2018100221062434,Tue Oct 02 21:06:24 CDT 2018,NB,I-35W,@ University Ave,ROADWORK
9,9,L004_9946465,Tue Oct 02 22:39:42 CDT 2018,NB,T.H.47,NaN,CRASH
